In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [31]:
df = pd.read_csv('../Data/novel_sep.csv', lineterminator='\n', index_col=0)

In [32]:
df

,id,itemId,title,pubDate,coverSmallUrl,coverLargeUrl,publisher,priceStandard,customerReviewRank,author,isbn,link,story,review,piece,analysis
0,1,349572433,주술회전 1: 떠나가는 여름과 돌아오는 가을,20210525,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,서울미디어코믹스(서울문화사),7800,0.0,Gege Akutami(아쿠타미 제제),9791136734518,http://book.interpark.com/blog/integration/pro...,"대인기 다크 액션 판타지, 「주술회전」소설 1권 발행!!이타도리와 후시구로의 아키하...",NaN,NaN,인기 다크 액션 판타지 주술 회전 발행 도리 후시 구로 아키하바라 산책 고 나 나미...
1,2,348109712,흐르는 강물처럼(40주년 기념판),20210415,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,연암서가,15000,0.0,노먼 매클린,9791160870763,http://book.interpark.com/blog/integration/pro...,영화 〈흐르는 강물처럼〉의 원작소설영화보다 더한 감동이 굽이쳐 흐른다! 〈흐르는 강...,NaN,나는 1981년 몬태나 주를 방문하던 중에 내 친구 토머스 맥구언과 함께 미국 서부...,영화 흐르는 강물 영화 더한 감동 굽이쳐 흐른다 흐르는 강물 감독 로버트 레드 포드...
2,3,340692027,백작가의 비밀스런 시녀님 세트,20201027,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,필프리미엄에디션,38000,8.0,백주아,9791165655778,http://book.interpark.com/blog/integration/pro...,백주아 장편소설 『백작가의 비밀스런 시녀님 세트』. 가난한 소작농의 딸 폴라. 우연...,〈1권〉가난한 소작농의 딸 폴라.우연한 계기로 명망 높은 벨루니타 백작가의 사용인으...,NaN,백주 비밀스런 녀 세트 가난한 소작농 딸 폴라 우연 계기 명망 높은 벨루 타 사용 ...
3,4,343464716,신세계에서 2,20201125,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,해냄출판사,18000,9.9,기시 유스케(貴志祐介),9788965740322,http://book.interpark.com/blog/integration/pro...,일본 SF대상 수상작 & 서점대상 제6위미래의 가상세계에 빗대어 현 인류의 모순을 ...,"순백의 아이들이 평화롭게 살아가는 머나먼 미래, 그 아름다운 유토피아에 가려진 잔혹...",X. 얼굴 없는 소년. 누구보다 사랑했던 소년. 그런데 지금은 얼굴도 떠올릴 수 없...,일본 대상 서점 대상 미래 가상세계 빗대어 현 인류 모순 전면 드러낸 기시 유스케 ...
4,5,340768143,추리소설가의 살인사건,20201028,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,소미미디어,15800,9.5,히가시노 게이고(東野圭吾),9791166111983,http://book.interpark.com/blog/integration/pro...,"추리 소설가, 편집자, 독자가 등장하는 추리소설,히가시노 게이고만이 쓸 수 있는 색...","이 작품집, 꽤나 웃기다! 한 번 읽기 시작했다면 멈출 수 없다! 올해의 페이지 터...",그것은 하마사키 회계사무소에서 온 서류였다. 소장인 하마사키 고로는 내 고등학교 동...,추리 편집자 등장 추리소설 가시노 게이고 쓸 색다른 블랙 코미디 대중교통 읽지 마시...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1113,1114,345966624,어니언 마말레이드 2,20210121,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,동아,12800,0.0,백서하,9791163024453,http://book.interpark.com/blog/integration/pro...,"사랑에 빠진 위그의 뒤통수를 치고,이디에트 가문의 약점을 손에 쥐게 된 비비안. “...",NaN,“상속법 관련 문제에 대해 의견을 묻고 싶네.”“죄송하지만 그 부분에 대해서는 귀족...,사랑 빠진 뒤통수 디 트 가문 약점 손 쥐 비비안 전략 사랑 순수하게 날 사랑 아니...
1114,1115,343789806,갑의 품격 1,20201130,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,가연,14500,0.0,김나든,9788968970818,http://book.interpark.com/blog/integration/pro...,"네이버 시리즈 다운로드 수 84만, 실시간 랭킹 1위.혜성처럼 나타난 김나든 작가의...","서울의 끝자락 판자촌 태생, 부모가 누군지도 모르는 고아. 보육원을 떠돌던 예일은 ...","생전 처음 보는 사람 앞에서 그렇게 예일은 펑펑 울었다. 산발이 된 머리도, 그에게...",네이버 다운로드 실시간 랭킹 혜성 나타난 김나든 유쾌하면서 감동 갑 품격 웹툰 확정...
1115,1116,342291271,진실에 갇힌 남자,20201109,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,북로드,14800,9.5,데이비드 발다치,9791158791490,http://book.interpark.com/blog/integration/pro...,“진실이 늘 우리를 자유롭게 하는 것은 아니에요.때론 우리를 가두는 감옥이 될 수도...,"죽음을 눈앞에 두고 자신의 무죄를 주장하던 남자, 이미 죽은 것이나 다름없는 그가 ...","데커는 언젠가 땅 밑에 누워 있는 가족에게 자신도 합류하게 될 그날까지, 매년 몰리...",진실 늘 자유롭게 아니에요 때론 가두는 감옥 수도 있죠 아마존 뉴욕 타임스 기억 불...
1116,1117,342332559,검사 김서진 4,20201008,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,로크미디어,8000,0.0,이해날,9791135485893,http://book.interpark.com/blog/integration/pro...,No방심 작가 이해날의 검사물 리턴즈!그 검사의 목숨을 건 외줄 타기가 시작된다! ...,NaN,NaN,방심 이해 날 검사 물 리턴즈 검사 목숨 건 외줄 타기가 시작 차기 검찰총장 김영준...


In [4]:
df.columns

Index(['id', 'itemId', 'title', 'pubDate', 'coverSmallUrl', 'coverLargeUrl',
       'publisher', 'priceStandard', 'customerReviewRank', 'author', 'isbn',
       'link', 'story', 'review', 'piece', 'analysis'],
      dtype='object')

In [5]:
import gensim
gensim.__version__

'3.8.3'

In [6]:
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel
from gensim.test.utils import common_corpus
from gensim.models.wrappers import LdaMallet
import pyLDAvis
import pyLDAvis.gensim_models

In [7]:
# vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
# pyLDAvis.display(vis)
# pyLDAvis.enable_notebook()

In [8]:
df.drop_duplicates('story', inplace=True)

In [9]:
df

,id,itemId,title,pubDate,coverSmallUrl,coverLargeUrl,publisher,priceStandard,customerReviewRank,author,isbn,link,story,review,piece,analysis
0,1,349572433,주술회전 1: 떠나가는 여름과 돌아오는 가을,20210525,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,서울미디어코믹스(서울문화사),7800,0.0,Gege Akutami(아쿠타미 제제),9791136734518,http://book.interpark.com/blog/integration/pro...,"대인기 다크 액션 판타지, 「주술회전」소설 1권 발행!!이타도리와 후시구로의 아키하...",NaN,NaN,인기 다크 액션 판타지 주술 회전 소설 권 발행 도리 후시 구로 아키하바라 산책 고...
1,2,348109712,흐르는 강물처럼(40주년 기념판),20210415,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,연암서가,15000,0.0,노먼 매클린,9791160870763,http://book.interpark.com/blog/integration/pro...,영화 〈흐르는 강물처럼〉의 원작소설영화보다 더한 감동이 굽이쳐 흐른다! 〈흐르는 강...,NaN,나는 1981년 몬태나 주를 방문하던 중에 내 친구 토머스 맥구언과 함께 미국 서부...,영화 흐르는 강물 의 소설 영화 더한 감동 굽이쳐 흐른다 흐르는 강물 의 감독 로버...
2,3,340692027,백작가의 비밀스런 시녀님 세트,20201027,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,필프리미엄에디션,38000,8.0,백주아,9791165655778,http://book.interpark.com/blog/integration/pro...,백주아 장편소설 『백작가의 비밀스런 시녀님 세트』. 가난한 소작농의 딸 폴라. 우연...,〈1권〉가난한 소작농의 딸 폴라.우연한 계기로 명망 높은 벨루니타 백작가의 사용인으...,NaN,백주 장편소설 작가 비밀스런 녀 세트 가난한 소작농 딸 폴라 우연 계기 명망 높은 ...
3,4,343464716,신세계에서 2,20201125,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,해냄출판사,18000,9.9,기시 유스케(貴志祐介),9788965740322,http://book.interpark.com/blog/integration/pro...,일본 SF대상 수상작 & 서점대상 제6위미래의 가상세계에 빗대어 현 인류의 모순을 ...,"순백의 아이들이 평화롭게 살아가는 머나먼 미래, 그 아름다운 유토피아에 가려진 잔혹...",X. 얼굴 없는 소년. 누구보다 사랑했던 소년. 그런데 지금은 얼굴도 떠올릴 수 없...,일본 대상 수상작 서점 대상 제 위 미래 가상세계 빗대어 현 인류 모순 전면 드러낸...
4,5,340768143,추리소설가의 살인사건,20201028,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,소미미디어,15800,9.5,히가시노 게이고(東野圭吾),9791166111983,http://book.interpark.com/blog/integration/pro...,"추리 소설가, 편집자, 독자가 등장하는 추리소설,히가시노 게이고만이 쓸 수 있는 색...","이 작품집, 꽤나 웃기다! 한 번 읽기 시작했다면 멈출 수 없다! 올해의 페이지 터...",그것은 하마사키 회계사무소에서 온 서류였다. 소장인 하마사키 고로는 내 고등학교 동...,추리 소설가 편집자 독자 등장 추리소설 가시노 게이고 쓸 수 있는 색다른 블랙 코미...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1113,1114,345966624,어니언 마말레이드 2,20210121,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,동아,12800,0.0,백서하,9791163024453,http://book.interpark.com/blog/integration/pro...,"사랑에 빠진 위그의 뒤통수를 치고,이디에트 가문의 약점을 손에 쥐게 된 비비안. “...",NaN,“상속법 관련 문제에 대해 의견을 묻고 싶네.”“죄송하지만 그 부분에 대해서는 귀족...,사랑 빠진 위 의 뒤통수 디 트 가문 약점 손 쥐 비비안 전략 사랑 순수하게 날 사...
1114,1115,343789806,갑의 품격 1,20201130,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,가연,14500,0.0,김나든,9788968970818,http://book.interpark.com/blog/integration/pro...,"네이버 시리즈 다운로드 수 84만, 실시간 랭킹 1위.혜성처럼 나타난 김나든 작가의...","서울의 끝자락 판자촌 태생, 부모가 누군지도 모르는 고아. 보육원을 떠돌던 예일은 ...","생전 처음 보는 사람 앞에서 그렇게 예일은 펑펑 울었다. 산발이 된 머리도, 그에게...",네이버 시리즈 다운로드 수 실시간 랭킹 위 혜성 나타난 김나든 작가 유쾌하면서 감동...
1115,1116,342291271,진실에 갇힌 남자,20201109,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,북로드,14800,9.5,데이비드 발다치,9791158791490,http://book.interpark.com/blog/integration/pro...,“진실이 늘 우리를 자유롭게 하는 것은 아니에요.때론 우리를 가두는 감옥이 될 수도...,"죽음을 눈앞에 두고 자신의 무죄를 주장하던 남자, 이미 죽은 것이나 다름없는 그가 ...","데커는 언젠가 땅 밑에 누워 있는 가족에게 자신도 합류하게 될 그날까지, 매년 몰리...",진실 늘 우리 자유롭게 것 아니에요 때론 우리 가두는 감옥 수도 있죠 아마존 베스트...
1116,1117,342332559,검사 김서진 4,20201008,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,로크미디어,8000,0.0,이해날,9791135485893,http://book.interpark.com/blog/integration/pro...,No방심 작가 이해날의 검사물 리턴즈!그 검사의 목숨을 건 외줄 타기가 시작된다! ...,NaN,NaN,방심 작가 이해 날 검사 물 리턴즈 그 검사 목숨 건 외줄 타기가 시작 차기 검찰총...


In [10]:
df.fillna('',inplace=True)
df.reset_index(drop=True, inplace=True)

In [11]:
df

,id,itemId,title,pubDate,coverSmallUrl,coverLargeUrl,publisher,priceStandard,customerReviewRank,author,isbn,link,story,review,piece,analysis
0,1,349572433,주술회전 1: 떠나가는 여름과 돌아오는 가을,20210525,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,서울미디어코믹스(서울문화사),7800,0.0,Gege Akutami(아쿠타미 제제),9791136734518,http://book.interpark.com/blog/integration/pro...,"대인기 다크 액션 판타지, 「주술회전」소설 1권 발행!!이타도리와 후시구로의 아키하...",,,인기 다크 액션 판타지 주술 회전 소설 권 발행 도리 후시 구로 아키하바라 산책 고...
1,2,348109712,흐르는 강물처럼(40주년 기념판),20210415,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,연암서가,15000,0.0,노먼 매클린,9791160870763,http://book.interpark.com/blog/integration/pro...,영화 〈흐르는 강물처럼〉의 원작소설영화보다 더한 감동이 굽이쳐 흐른다! 〈흐르는 강...,,나는 1981년 몬태나 주를 방문하던 중에 내 친구 토머스 맥구언과 함께 미국 서부...,영화 흐르는 강물 의 소설 영화 더한 감동 굽이쳐 흐른다 흐르는 강물 의 감독 로버...
2,3,340692027,백작가의 비밀스런 시녀님 세트,20201027,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,필프리미엄에디션,38000,8.0,백주아,9791165655778,http://book.interpark.com/blog/integration/pro...,백주아 장편소설 『백작가의 비밀스런 시녀님 세트』. 가난한 소작농의 딸 폴라. 우연...,〈1권〉가난한 소작농의 딸 폴라.우연한 계기로 명망 높은 벨루니타 백작가의 사용인으...,,백주 장편소설 작가 비밀스런 녀 세트 가난한 소작농 딸 폴라 우연 계기 명망 높은 ...
3,4,343464716,신세계에서 2,20201125,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,해냄출판사,18000,9.9,기시 유스케(貴志祐介),9788965740322,http://book.interpark.com/blog/integration/pro...,일본 SF대상 수상작 & 서점대상 제6위미래의 가상세계에 빗대어 현 인류의 모순을 ...,"순백의 아이들이 평화롭게 살아가는 머나먼 미래, 그 아름다운 유토피아에 가려진 잔혹...",X. 얼굴 없는 소년. 누구보다 사랑했던 소년. 그런데 지금은 얼굴도 떠올릴 수 없...,일본 대상 수상작 서점 대상 제 위 미래 가상세계 빗대어 현 인류 모순 전면 드러낸...
4,5,340768143,추리소설가의 살인사건,20201028,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,소미미디어,15800,9.5,히가시노 게이고(東野圭吾),9791166111983,http://book.interpark.com/blog/integration/pro...,"추리 소설가, 편집자, 독자가 등장하는 추리소설,히가시노 게이고만이 쓸 수 있는 색...","이 작품집, 꽤나 웃기다! 한 번 읽기 시작했다면 멈출 수 없다! 올해의 페이지 터...",그것은 하마사키 회계사무소에서 온 서류였다. 소장인 하마사키 고로는 내 고등학교 동...,추리 소설가 편집자 독자 등장 추리소설 가시노 게이고 쓸 수 있는 색다른 블랙 코미...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1113,1114,345966624,어니언 마말레이드 2,20210121,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,동아,12800,0.0,백서하,9791163024453,http://book.interpark.com/blog/integration/pro...,"사랑에 빠진 위그의 뒤통수를 치고,이디에트 가문의 약점을 손에 쥐게 된 비비안. “...",,“상속법 관련 문제에 대해 의견을 묻고 싶네.”“죄송하지만 그 부분에 대해서는 귀족...,사랑 빠진 위 의 뒤통수 디 트 가문 약점 손 쥐 비비안 전략 사랑 순수하게 날 사...
1114,1115,343789806,갑의 품격 1,20201130,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,가연,14500,0.0,김나든,9788968970818,http://book.interpark.com/blog/integration/pro...,"네이버 시리즈 다운로드 수 84만, 실시간 랭킹 1위.혜성처럼 나타난 김나든 작가의...","서울의 끝자락 판자촌 태생, 부모가 누군지도 모르는 고아. 보육원을 떠돌던 예일은 ...","생전 처음 보는 사람 앞에서 그렇게 예일은 펑펑 울었다. 산발이 된 머리도, 그에게...",네이버 시리즈 다운로드 수 실시간 랭킹 위 혜성 나타난 김나든 작가 유쾌하면서 감동...
1115,1116,342291271,진실에 갇힌 남자,20201109,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,북로드,14800,9.5,데이비드 발다치,9791158791490,http://book.interpark.com/blog/integration/pro...,“진실이 늘 우리를 자유롭게 하는 것은 아니에요.때론 우리를 가두는 감옥이 될 수도...,"죽음을 눈앞에 두고 자신의 무죄를 주장하던 남자, 이미 죽은 것이나 다름없는 그가 ...","데커는 언젠가 땅 밑에 누워 있는 가족에게 자신도 합류하게 될 그날까지, 매년 몰리...",진실 늘 우리 자유롭게 것 아니에요 때론 우리 가두는 감옥 수도 있죠 아마존 베스트...
1116,1117,342332559,검사 김서진 4,20201008,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,로크미디어,8000,0.0,이해날,9791135485893,http://book.interpark.com/blog/integration/pro...,No방심 작가 이해날의 검사물 리턴즈!그 검사의 목숨을 건 외줄 타기가 시작된다! ...,,,방심 작가 이해 날 검사 물 리턴즈 그 검사 목숨 건 외줄 타기가 시작 차기 검찰총...


## LDA 함수 만들어놓기

In [27]:
def lda_coh(stories, no_below=5, num_topics=4, data_word=None, data_words=None):
    
    # 이야기 전체 형태소 분리
    if data_word == None:
        data_words = []
        for story in stories:
            data = list(str(story).split())
            data_words.append(data)
        
    
    if data_word == 'stop':
        data_words = []
        for story in stories:
            data = list(str(story).split())
            data_words.append(data)
            
        return data_words
    
    # id2word, corpus 추출
    id2word = corpora.Dictionary(data_words)
    id2word.filter_extremes(no_below = no_below) # no_below 회 이하로 등장한 단어는 삭제
    texts = data_words
    corpus = [id2word.doc2bow(text) for text in texts]
    
    mallet_path = '../mallet-2.0.8/bin/mallet' 
    ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, random_seed=12, id2word=id2word)

    coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_ldamallet = coherence_model_ldamallet.get_coherence()
    
    return texts, corpus, ldamallet, coherence_ldamallet

In [28]:
def lda_coh_id2(stories, no_below=5, num_topics=4, data_word=None, data_words=None):
    
    # 이야기 전체 형태소 분리
    if data_word == None:
        data_words = []
        for story in stories:
            data = list(str(story).split())
            data_words.append(data)
        
    
    if data_word == 'stop':
        data_words = []
        for story in stories:
            data = list(str(story).split())
            data_words.append(data)
            
        return data_words
    
    # id2word, corpus 추출
    id2word = corpora.Dictionary(data_words)
    id2word.filter_extremes(no_below = no_below) # no_below 회 이하로 등장한 단어는 삭제
    texts = data_words
    corpus = [id2word.doc2bow(text) for text in texts]
    
    mallet_path = '../mallet-2.0.8/bin/mallet' 
    ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, random_seed=12, id2word=id2word)

    coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_ldamallet = coherence_model_ldamallet.get_coherence()
    
    return texts, corpus, ldamallet, coherence_ldamallet, id2word

In [29]:
def format_topics_sentences(texts, corpus, data_words, ldamallet, Data=df):
    # Init output

    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    #ldamallet[corpus]: lda_model에 corpus를 넣어 각 토픽 당 확률을 알 수 있음
    for i, row in enumerate(ldamallet[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamallet.show_topic(topic_num, topn=10)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    print(type(sent_topics_df))

    # Add original text to the end of the output
    contents = pd.Series(data_words)
    # sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    sent_topics_df = pd.concat([Data['title'], Data['story'], sent_topics_df], axis=1)

    return sent_topics_df

In [30]:
def compute_coherence_values(stories, limit=10, start=4, step=2):

    from tqdm import tqdm
    coherence_values = []
    model_list = []
    for num_topics in tqdm(range(start, limit, step)):
        data_words = lda_coh(stories, data_word='stop')
        texts, corpus, ldamallet, coherence_ldamallet = lda_coh(stories, no_below=5, num_topics=num_topics, data_words=data_words)
        model_list.append(ldamallet)
        coherence_values.append(coherence_ldamallet)
        
    return model_list, coherence_values

### 스토리 나눠서 분석해보기

## 1. 책소개 (story) 

In [16]:
stories = df['story']
# df['story'] + df['review'] + df['piece']

In [17]:
limit=20; start=4; step=2;

model_list, coherence_values = compute_coherence_values(stories, limit=limit, start=start, step=step)

  0%|          | 0/8 [00:00<?, ?it/s]


CalledProcessError: Command '../mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input /var/folders/hr/y8znbyhd4058w4d5l14qqsv80000gn/T/c2c694_corpus.txt --output /var/folders/hr/y8znbyhd4058w4d5l14qqsv80000gn/T/c2c694_corpus.mallet' returned non-zero exit status 127.

In [ ]:
x = range(start, limit, step)

find_model = {}
for m,c in zip(x, coherence_values):
    find_model[m] = c
print(find_model)

### 모델 검증 결과 시각화

In [ ]:
x = find_model.keys()
y = find_model.values()

plt.plot(x, y)
plt.show()

In [ ]:
max(find_model.values())

* 12개로 나눴을 때 가장 높은 점수를 보임. (0.32318150117756983)

## 2. 서평 (review) 

In [ ]:
stories = df['review']
# df['story'] + df['review'] + df['piece']

In [ ]:
limit=20; start=4; step=2;

model_list, coherence_values = compute_coherence_values(stories, limit=limit, start=start, step=step)

In [ ]:
x = range(start, limit, step)

find_model = {}
for m,c in zip(x, coherence_values):
    find_model[m] = c
print(find_model)

### 모델 검증 결과 시각화

In [ ]:
x = find_model.keys()
y = find_model.values()

plt.plot(x, y)
plt.show()

In [ ]:
max(find_model.values())

* 10개로 나눴을 때 가장 높은 점수를 보임. (0.4090067934353466)

## 3. 책내용 일부 (piece) 

In [ ]:
stories = df['piece']
# df['story'] + df['review'] + df['piece']

In [ ]:
limit=20; start=4; step=2;

model_list, coherence_values = compute_coherence_values(stories, limit=limit, start=start, step=step)

In [ ]:
x = range(start, limit, step)

find_model = {}
for m,c in zip(x, coherence_values):
    find_model[m] = c
print(find_model)

### 모델 검증 결과 시각화

In [ ]:
x = find_model.keys()
y = find_model.values()

plt.plot(x, y)
plt.show()

In [ ]:
max(find_model.values())

* 4개로 나눴을 때 가장 높은 점수를 보임. (0.3385571328156868)

## 4. 책소개 (story) + 서평 (review)

In [ ]:
stories = df['story'] + df['review']
# df['story'] + df['review'] + df['piece']

In [ ]:
limit=20; start=4; step=2;

model_list, coherence_values = compute_coherence_values(stories, limit=limit, start=start, step=step)

In [ ]:
x = range(start, limit, step)

find_model = {}
for m,c in zip(x, coherence_values):
    find_model[m] = c
print(find_model)

### 모델 검증 결과 시각화

In [ ]:
x = find_model.keys()
y = find_model.values()

plt.plot(x, y)
plt.show()

In [ ]:
max_model = max(find_model,key=find_model.get)
texts, corpus, ldamallet, coherence_ldamallet = lda_coh(stories, num_topics=max_model)
data_words = lda_coh(stories, num_topics=max_model, data_word='stop')
df_topic_sents_keywords = format_topics_sentences(texts, corpus, data_words, ldamallet, Data=df)

In [ ]:
max(find_model.values())

* 10개로 나눴을 때 가장 높은 점수를 보임. (0.4201821884276328)

## 5. 서평 (review) + 책내용 일부 (piece)

In [ ]:
stories = df['review']
# df['story'] + df['review'] + df['piece']

In [ ]:
limit=20; start=4; step=2;

model_list, coherence_values = compute_coherence_values(stories, limit=limit, start=start, step=step)

In [ ]:
x = range(start, limit, step)

find_model = {}
for m,c in zip(x, coherence_values):
    find_model[m] = c
print(find_model)

### 모델 검증 결과 시각화

In [ ]:
x = find_model.keys()
y = find_model.values()

plt.plot(x, y)
plt.show()

In [ ]:
max(find_model.values())

* 10개로 나눴을 때 가장 높은 점수를 보임. (0.4090067934353466)

## 6. 책소개 (story) + 책내용 일부 (piece) 

In [ ]:
stories = df['piece']
# df['story'] + df['review'] + df['piece']

In [ ]:
limit=20; start=4; step=2;

model_list, coherence_values = compute_coherence_values(stories, limit=limit, start=start, step=step)

In [ ]:
x = range(start, limit, step)

find_model = {}
for m,c in zip(x, coherence_values):
    find_model[m] = c
print(find_model)

### 모델 검증 결과 시각화

In [ ]:
x = find_model.keys()
y = find_model.values()

plt.plot(x, y)
plt.show()

In [ ]:
max(find_model.values())

* 4개로 나눴을 때 가장 높은 점수를 보임. (0.3385571328156868)

## 7. 책소개 (story) + 책리뷰 (review) +  책내용 일부 (piece) 

In [ ]:
stories = df['piece'] + df['story'] + df['review'] 
# df['story'] + df['review'] + df['piece']

In [ ]:
limit=20; start=4; step=2;

model_list, coherence_values = compute_coherence_values(stories, limit=limit, start=start, step=step)

In [ ]:
x = range(start, limit, step)

find_model = {}
for m,c in zip(x, coherence_values):
    find_model[m] = c
print(find_model)

### 모델 검증 결과 시각화

In [ ]:
x = find_model.keys()
y = find_model.values()

plt.plot(x, y)
plt.show()

In [ ]:
max(find_model.values())

* 10개로 나눴을 때 가장 높은 점수를 보임. (0.40895756440663134)

---

## 선택된 모델(책소개, 서평 / 10개 분류)로 토픽분류 시각화

In [33]:
stories = df['analysis']
# df['story'] + df['review'] + df['piece']

In [34]:
stories

0       인기 다크 액션 판타지 주술 회전 발행 도리 후시 구로 아키하바라 산책 고 나 나미...
1       영화 흐르는 강물 영화 더한 감동 굽이쳐 흐른다 흐르는 강물 감독 로버트 레드 포드...
2       백주 비밀스런 녀 세트 가난한 소작농 딸 폴라 우연 계기 명망 높은 벨루 타 사용 ...
3       일본 대상 서점 대상 미래 가상세계 빗대어 현 인류 모순 전면 드러낸 기시 유스케 ...
4       추리 편집자 등장 추리소설 가시노 게이고 쓸 색다른 블랙 코미디 대중교통 읽지 마시...
                              ...                        
1113    사랑 빠진 뒤통수 디 트 가문 약점 손 쥐 비비안 전략 사랑 순수하게 날 사랑 아니...
1114    네이버 다운로드 실시간 랭킹 혜성 나타난 김나든 유쾌하면서 감동 갑 품격 웹툰 확정...
1115    진실 늘 자유롭게 아니에요 때론 가두는 감옥 수도 있죠 아마존 뉴욕 타임스 기억 불...
1116    방심 이해 날 검사 물 리턴즈 검사 목숨 건 외줄 타기가 시작 차기 검찰총장 김영준...
1117    사회 공격 소시민 위로 받을 불안 분노 피해 의식 본다 박완서 단정한 뚫고 터져 나...
Name: analysis, Length: 1118, dtype: object

In [35]:
texts, corpus, ldamallet, coherence_ldamallet, id2word = lda_coh_id2(stories, num_topics=10)
data_words = lda_coh(stories, num_topics=10, data_word='stop')
df_topic_sents_keywords = format_topics_sentences(texts, corpus, data_words, ldamallet, Data=df)

/var/folders/hr/y8znbyhd4058w4d5l14qqsv80000gn/T/ipykernel_59022/1524641126.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
/var/folders/hr/y8znbyhd4058w4d5l14qqsv80000gn/T/ipykernel_59022/1524641126.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)


<class 'pandas.core.frame.DataFrame'>


In [36]:
from gensim.models import LdaModel

def mallet_to_lda(optimal_model):
   model_gensim = LdaModel(
       id2word=optimal_model.id2word, num_topics=optimal_model.num_topics,
       alpha=optimal_model.alpha, eta=0, iterations=1000,
       gamma_threshold=0.001,
       dtype=np.float32
   )
   model_gensim.sync_state()
   model_gensim.state.sstats = optimal_model.wordtopics

   return model_gensim

optimal_model = ldamallet
model = mallet_to_lda(optimal_model)

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
vis = gensimvis.prepare(model, corpus, id2word)

pyLDAvis.save_html(vis, './소설줄거리_LDA.html')
vis

/opt/homebrew/Caskroom/miniforge/base/envs/multi_crawling/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/opt/homebrew/Caskroom/miniforge/base/envs/multi_crawling/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/opt/homebrew/Caskroom/miniforge/base/envs/multi_crawling/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.029272 -0.126900       1        1  10.575000
3     -0.070699 -0.257112       2        1  10.477310
1      0.229930  0.118982       3        1  10.285152
5      0.107630  0.041710       4        1  10.203308
0     -0.258788  0.164369       5        1  10.108807
9      0.155977 -0.106018       6        1   9.879889
4     -0.196405  0.100504       7        1   9.693049
6      0.101192  0.179122       8        1   9.660548
7     -0.111948 -0.126913       9        1   9.607085
8      0.013838  0.012256      10        1   9.509853, topic_info=     Term         Freq        Total Category  logprob  loglift
471    아이  1458.000000  1458.000000  Default  30.0000  30.0000
16     사건  1375.000000  1375.000000  Default  29.0000  29.0000
858    그녀  1541.000000  1541.000000  Default  28.0000  28.0000
92     세계  2026.000000  2026.000000  Default  27.0000  27.0000
1445  청소년   906.000000   906.000000  Default  26.0000  26.0000
...   ...          ...          ...      ...      ...      ...
1095   사실   172.413171   817.620936  Topic10  -5.5864   0.7963
399    등장   155.267110   526.102267  Topic10  -5.6911   1.1325
109    영화   146.694079   417.004532  Topic10  -5.7479   1.3081
2430   매력   143.836403   378.530218  Topic10  -5.7676   1.3852
517    인간   135.263372  1817.444334  Topic10  -5.8291  -0.2451

[926 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
9487     10  0.997962    가가
154       2  1.005852    가명
4554      4  0.999730    가문
348       9  0.995943  가상세계
3035      1  0.469321    가슴
...     ...       ...   ...
150       4  0.070467     힘
150       5  0.026220     힘
150       6  0.362165     힘
150       7  0.090132     힘
150       9  0.065550     힘

[1537 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 2, 6, 1, 10, 5, 7, 8, 9])

In [23]:
# Format
df_topic_story = df_topic_sents_keywords

# 각 문서에 대한 토픽 내림차순
df_topic_sort=df_topic_story.sort_values(by=['Dominant_Topic'])

In [24]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)


In [25]:
sent_topics_sorteddf_mallet[['Perc_Contribution', 'Dominant_Topic', 'Topic_Keywords']]

,Perc_Contribution,Dominant_Topic,Topic_Keywords
478,0.9021,0,"한국, 사회, 시대, 국문학, 전집, 번역, 대표, 수록, 박완서, 새로운"
472,0.6961,1,"마음, 시간, 서로, 두, 감정, 세상, 관계, 살, 지금, 상처"
991,0.8261,2,"죽음, 그녀, 아버지, 소녀, 아들, 남자, 인생, 두, 운명, 딸"
924,0.9404,3,"역사, 시대, 일본, 가장, 권, 전쟁, 이자, 화, 임꺽정, 중국"
1050,0.7364,4,"내, 생각, 그녀, 눈, 더, 쪽, 몸, 있었다, 그런, 곳"
172,0.7507,5,"사건, 미스터리, 진실, 전, 위, 베스트셀러, 살인, 두, 일본, 스릴러"
473,0.6516,6,"인간, 세계, 미국, 를, 인류, 가장, 이자, 프랑스, 주인공, 세기"
864,0.9447,7,"권, 왕, 온, 로맨스, 드라마, 판타지, 제, 여인, 조선, 위"
1052,0.6569,8,"아이, 청소년, 가족, 친구, 학교, 엄마, 주인공, 꿈, 문제, 소년"
17,0.5409,9,"집, 세계, 여성, 존재, 인물, 한국, 를, 현실, 김, 소설가"


In [26]:
df_topic_story.value_counts('Dominant_Topic').sort_index()

Dominant_Topic
0     67
1    109
2     98
3     72
4    112
5    156
6    130
7    109
8    138
9    127
dtype: int64

In [ ]:
df_topic_story.to_csv('소설_책내용_서평기반_LDA_완료.csv')

In [ ]:
df_topic_story